In [43]:
!pip install mlflow
!pip install pyngrok

In [44]:
from pyngrok import ngrok

# Kill all active tunnels
ngrok.kill()


In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("Exp 2- Bow Vs Tfidf")

from google.colab import userdata
ngRoktoken=userdata.get('ngroktoen')

get_ipython().system_raw('mlflow ui --port 2000 &')
mlflow.set_tracking_uri('http://localhost:2000')
from pyngrok import ngrok
ngrok.set_auth_token(ngRoktoken)

public_url=ngrok.connect(2000).public_url
print('mlflow ui url: ', public_url)

2025/06/28 05:31:17 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2- Bow Vs Tfidf' does not exist. Creating a new experiment.


mlflow ui url:  https://f263-35-245-197-222.ngrok-free.app


In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
df=pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [ ]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

# Assume df is already defined and has columns 'clean_comment' and 'category'

# step 1: function to run the experiments
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
  # step 2: vectorization
  if vectorizer_type == 'Bow':
    vectorize = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
  else:
    vectorize = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

  X = vectorize.fit_transform(df['clean_comment'])
  y = df['category']

  # step 3: train test split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

  # step 4: define and train a random forest model
  run_name = f'{vectorizer_name}_{ngram_range}_randomforest'
  with mlflow.start_run(run_name=run_name) as run:
    # set tags for the experiment and run
    mlflow.set_tag('experiment_type', 'feature_engineering')
    mlflow.set_tag('model_type', 'randomforestclassifier')

    # add a description
    mlflow.set_tag('description', f'randomforest with {vectorizer_name} , ngram_range={ngram_range},max_features={vectorizer_max_features}')

    # log vectorizer parameters
    mlflow.log_param('vectorizer_type', vectorizer_type)
    mlflow.log_param('ngram_range', str(ngram_range))
    mlflow.log_param('vectorizer_max_features', vectorizer_max_features)

    # log random forest parameters
    n_estimators = 200
    max_depth = 15
    mlflow.log_param('n_estimators', n_estimators)
    mlflow.log_param('max_depth', max_depth)

    # initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # step 5: make predictions and log metrics
    y_pred = model.predict(X_test)

    # Log accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric('accuracy', accuracy)

    # log classification report
    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
      if isinstance(metrics, dict):
        for metric, value in metrics.items():
          mlflow.log_metric(f'{label}_{metric}', value)

    # log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('prediction')
    plt.ylabel('actual')
    plt.title(f'confusion matrix: {vectorizer_name}, {ngram_range}')
    file_name = f'confusion_matrix_{vectorizer_name}_{ngram_range[0]}_{ngram_range[1]}.png'
    plt.savefig(file_name)
    mlflow.log_artifact(file_name)
    plt.close()

    # log the model with signature and input_example
    signature = infer_signature(X_test, y_pred)
    mlflow.sklearn.log_model(
        sk_model=model,
        name=f'random_forest_model_{vectorizer_name}_{ngram_range}',
        input_example=X_test[:1],
        signature=signature
    )

# step 6: run the experiment for BoW and TF-IDF with different n-gram
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigram, bi, and tri
max_features = 5000  # example max feature size

for ngram_range in ngram_ranges:
  # BoW Experiments
  run_experiment('BoW', ngram_range, max_features, vectorizer_name='BoW')

  # TF-IDF Experiments
  run_experiment('TF-IDF', ngram_range, max_features, vectorizer_name='TF-IDF')


🏃 View run BoW_(1, 1)_randomforest at: http://localhost:2000/#/experiments/428468243532904247/runs/b2faad171cb341598ca37755bba2baa5
🧪 View experiment at: http://localhost:2000/#/experiments/428468243532904247
🏃 View run TF-IDF_(1, 1)_randomforest at: http://localhost:2000/#/experiments/428468243532904247/runs/5bf1ba71c3984e119f8256fc3b2d59e7
🧪 View experiment at: http://localhost:2000/#/experiments/428468243532904247
🏃 View run BoW_(1, 2)_randomforest at: http://localhost:2000/#/experiments/428468243532904247/runs/af6d17a7931e4d94ad7e19c39ef4b2c5
🧪 View experiment at: http://localhost:2000/#/experiments/428468243532904247
🏃 View run TF-IDF_(1, 2)_randomforest at: http://localhost:2000/#/experiments/428468243532904247/runs/736fbcbfd5904e9489dad5e41a1a25e4
🧪 View experiment at: http://localhost:2000/#/experiments/428468243532904247
🏃 View run BoW_(1, 3)_randomforest at: http://localhost:2000/#/experiments/428468243532904247/runs/8335b07b6075440cb188e9aa5b2815ba
🧪 View experiment at: http